# When people are discussing popular media, there’s a concept of spoilers. That is, critical information about the plot of a TV show, book, or movie that “ruins” the experience for people who haven’t read / seen it yet.

# Fortunately, the site TV Tropes has provided us with annotations of whether particular text is a spoiler or not (go to the page for your favorite TV show / movie and see what’s hidden).

## For the third assignment of my graduate course in machine learning, we had an in-class Kaggle competition to see who could get the best score on spoiler detection with feature engineering

In [1]:
import pandas as pd

## First we'll look at the original dataset we were given

In [2]:
original_train_data = pd.read_csv('spoiler_train_orig.csv')

In [3]:
original_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14784 entries, 0 to 14783
Data columns (total 5 columns):
sentence    14784 non-null object
spoiler     14784 non-null bool
verb        14784 non-null bool
page        14784 non-null object
trope       14784 non-null object
dtypes: bool(2), object(3)
memory usage: 375.5+ KB


In [4]:
pd.options.display.max_colwidth = 110

In [5]:
original_train_data.sentence[0:10]

0                                                                                     He  saw  me  at his surgery.
1                                              Repeats have been rare, but BBC 4 now shows the occasional episode.
2    Between his code of chivalry and a bad romantic past, Adam consistantly avoids the advances women, and the...
3          Adam drives a Mini Cooper S, a fairly humdrum compact car, but with an expensively remodelled interior.
4                                                   Later we discover that The Face had himself frozen after Adam.
5      Andromeda, though she works in conjuction with the computer rather than having it downloaded into her head.
6    Azaran, a newly-independent Middle Eastern country which hopes to use the computer to resolve its agricult...
7                                          While not stated in those terms, that is what Andromeda offers Fleming.
8    The show had its own  Spin-Off ,  ALF Tales , which had the characters as  

In [7]:
original_train_data.spoiler.value_counts()/original_train_data.spoiler.count()

True     0.527327
False    0.472673
Name: spoiler, dtype: float64

## Looks like we have a fairly balanced dataset. I think accuracy would be a good metric in this case

## Now let's look at the modified dataset that I created

In [8]:
modified_train_data = pd.read_csv('spoiler_train_modified.csv')

In [9]:
modified_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14784 entries, 0 to 14783
Data columns (total 10 columns):
group_tropes               14784 non-null object
sentence_pattern_parsed    14558 non-null object
genres                     14784 non-null object
contains_characters        14784 non-null object
separate_trope_words       14752 non-null object
trope                      14784 non-null object
sentence_subjectivity      14784 non-null object
sentence_polarity          14784 non-null object
sentence_modality          14784 non-null object
spoiler                    14784 non-null bool
dtypes: bool(1), object(9)
memory usage: 1.0+ MB


In [34]:
modified_train_data[0:10]

,group_tropes,sentence_pattern_parsed,genres,contains_characters,separate_trope_words,trope,sentence_subjectivity,sentence_polarity,sentence_modality,spoiler
0,unknown_trope,see,genre_Comedy,CONTAINS_CHARACTERS:FALSE,Comically Missing Point,ComicallyMissingThePoint,VERY_OBJECTIVE,NEGATIVE,FACTUAL,False
1,unknown_trope,repeat rare NNP show occasional,genre_Adventure Sci-Fi,CONTAINS_CHARACTERS:FALSE,Avengers.html,TheAvengers.html,SUBJECTIVE,POSITIVE,FACTUAL,False
2,unknown_trope,bad romantic NNP avoid interested work,genre_Adventure Sci-Fi,CONTAINS_CHARACTERS:TRUE,Celibate Hero,CelibateHero,OBJECTIVE,NEGATIVE,NOT_FACTUAL,False
3,unknown_trope,NNP drive NNP NNP NNP compact remodelled,genre_Adventure Sci-Fi,CONTAINS_CHARACTERS:TRUE,Cool Car,CoolCar,VERY_SUBJECTIVE,NEGATIVE,NOT_FACTUAL,False
4,narrative_device_trope,discover NNP freeze NNP,genre_Adventure Sci-Fi,CONTAINS_CHARACTERS:TRUE,Human Popsicle,HumanPopsicle,VERY_OBJECTIVE,NEGATIVE,FACTUAL,False
5,unknown_trope,NNP work download,genre_Action Adventure Drama Sci-Fi,CONTAINS_CHARACTERS:TRUE,Wetware Body,WetwareBody,VERY_OBJECTIVE,NEGATIVE,FACTUAL,False
6,unknown_trope,NNP newly-independent NNP NNP hope use resolve agricultural,genre_Action Adventure Drama Sci-Fi,CONTAINS_CHARACTERS:FALSE,Qurac,Qurac,VERY_OBJECTIVE,NEGATIVE,FACTUAL,False
7,drama_trope,state NNP offer NNP,genre_Action Adventure Drama Sci-Fi,CONTAINS_CHARACTERS:TRUE,Rule Together,WeCanRuleTogether,VERY_OBJECTIVE,NEGATIVE,FACTUAL,False
8,unknown_trope,NNP NNP animated perform modernized classic,genre_Comedy Drama Family Sci-Fi,CONTAINS_CHARACTERS:TRUE,Fun Acronyms,FunWithAcronyms,SUBJECTIVE,POSITIVE,FACTUAL,False
9,unknown_trope,comic tend play give strange technological seem inspire animated original would include set NNP framing NN...,genre_Comedy Drama Family Sci-Fi,CONTAINS_CHARACTERS:TRUE,Fun Acronyms,FunWithAcronyms,OBJECTIVE,POSITIVE,NOT_FACTUAL,False


## Here are the features I used:
- group_tropes: The trope category (which I obtained from TV Tropes)
- sentence_pattern_parsed: The sentence modified by the pattern library. I used stop-word removal, stemming, and replaced some words by their POS tag
- genres: The genre of the TV show or movie (obtained from IMDb)
- contains_characters: Indicates whether the sentence contains a character name.
- separate trope words: I separated the trope into individual words and removed stop words
- sentence_subjectivity: The subjectivity of the sentence, from VERY_SUBJECTIVE to VERY_OBJECTIVE
- sentence_polarity: The polarity of the sentence (NEGATIVE or POSITIVE)

In [35]:
modified_train_data.loc[modified_train_data.spoiler].group_tropes.value_counts()/modified_train_data.spoiler.sum()

unknown_trope                                                      0.634813
death_trope                                                        0.076065
narrative_device_trope                                             0.048615
plot_trope                                                         0.030144
spoilered_rotten_trope                                             0.028861
ending_trope                                                       0.021036
character_reaction_trope                                           0.017958
ending_trope spoilered_rotten_trope                                0.014495
drama_trope                                                        0.013725
action_adventure_trope                                             0.013084
betrayal_trope                                                     0.009364
action_adventure_trope spoilered_rotten_trope                      0.007440
plot_twist_trope                                                   0.004618
narrative_de

In [10]:
from sklearn.linear_model import SGDClassifier

In [11]:
modified_train_data.group_tropes.nunique()

93

In [12]:
top_tropes = ['death_trope', 'narrative_device_trope', 'plot_trope', 'spoilered_rotten_trope',
             'ending_trope', 'character_reaction_trope', 'character_reaction_trope', 
              'ending_trope spoilered_rotten_trope', 'drama_trope', 'action_adventure_trope',
             'betrayal_trope', 'action_adventure_trope spoilered_rotten_trope', 
             'plot_twist_trope']
modified_train_data.loc[~modified_train_data.group_tropes.isin(top_tropes), 'group_tropes'] = 'Other'

In [13]:
top_tropes_ohe = pd.get_dummies(modified_train_data.group_tropes)

In [14]:
modified_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14784 entries, 0 to 14783
Data columns (total 10 columns):
group_tropes               14784 non-null object
sentence_pattern_parsed    14558 non-null object
genres                     14784 non-null object
contains_characters        14784 non-null object
separate_trope_words       14752 non-null object
trope                      14784 non-null object
sentence_subjectivity      14784 non-null object
sentence_polarity          14784 non-null object
sentence_modality          14784 non-null object
spoiler                    14784 non-null bool
dtypes: bool(1), object(9)
memory usage: 1.0+ MB


In [15]:
characters_ohe = pd.get_dummies(modified_train_data.contains_characters)

In [16]:
subjectivity_ohe = pd.get_dummies(modified_train_data.sentence_subjectivity)

In [17]:
polarity_ohe = pd.get_dummies(modified_train_data.sentence_polarity)

In [18]:
modality_ohe = pd.get_dummies(modified_train_data.sentence_modality)

In [19]:
from functools import reduce

In [20]:
ohes = [top_tropes_ohe, characters_ohe, subjectivity_ohe, polarity_ohe, modality_ohe]

In [21]:
df_ohe = reduce(lambda left,right: pd.merge(left, right ,right_index=True, left_index=True), ohes)

In [22]:
df_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14784 entries, 0 to 14783
Data columns (total 23 columns):
Other                                            14784 non-null uint8
action_adventure_trope                           14784 non-null uint8
action_adventure_trope spoilered_rotten_trope    14784 non-null uint8
betrayal_trope                                   14784 non-null uint8
character_reaction_trope                         14784 non-null uint8
death_trope                                      14784 non-null uint8
drama_trope                                      14784 non-null uint8
ending_trope                                     14784 non-null uint8
ending_trope spoilered_rotten_trope              14784 non-null uint8
narrative_device_trope                           14784 non-null uint8
plot_trope                                       14784 non-null uint8
plot_twist_trope                                 14784 non-null uint8
spoilered_rotten_trope                           14

In [23]:
from sklearn.model_selection import train_test_split

In [26]:
labels = modified_train_data.spoiler
X_train, X_test, y_train, y_test = train_test_split(df_ohe, labels, test_size=0.2)

In [27]:
clf = SGDClassifier(loss='log', verbose=1, max_iter=5000, learning_rate='optimal')

In [28]:
clf.fit(X_train, y_train)

-- Epoch 1
Norm: 6.68, NNZs: 23, Bias: 2.576286, T: 11827, Avg. loss: 2.962612
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 3.96, NNZs: 23, Bias: 2.400200, T: 23654, Avg. loss: 0.993504
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 3.57, NNZs: 23, Bias: 2.596062, T: 35481, Avg. loss: 0.826611
Total training time: 0.01 seconds.
-- Epoch 4
Norm: 3.71, NNZs: 23, Bias: 2.435681, T: 47308, Avg. loss: 0.768701
Total training time: 0.01 seconds.
-- Epoch 5
Norm: 3.12, NNZs: 23, Bias: 2.324368, T: 59135, Avg. loss: 0.741547
Total training time: 0.01 seconds.
-- Epoch 6
Norm: 3.29, NNZs: 23, Bias: 2.197530, T: 70962, Avg. loss: 0.725733
Total training time: 0.01 seconds.
-- Epoch 7
Norm: 3.01, NNZs: 23, Bias: 2.058880, T: 82789, Avg. loss: 0.712984
Total training time: 0.02 seconds.
-- Epoch 8
Norm: 3.08, NNZs: 23, Bias: 1.935935, T: 94616, Avg. loss: 0.705690
Total training time: 0.02 seconds.
-- Epoch 9
Norm: 2.92, NNZs: 23, Bias: 2.148750, T: 106443, Avg. loss: 0.701700
Total tr

Norm: 2.60, NNZs: 23, Bias: 1.591188, T: 1099911, Avg. loss: 0.664615
Total training time: 0.20 seconds.
-- Epoch 94
Norm: 2.58, NNZs: 23, Bias: 1.575997, T: 1111738, Avg. loss: 0.664328
Total training time: 0.20 seconds.
-- Epoch 95
Norm: 2.64, NNZs: 23, Bias: 1.524629, T: 1123565, Avg. loss: 0.664666
Total training time: 0.21 seconds.
-- Epoch 96
Norm: 2.60, NNZs: 23, Bias: 1.552924, T: 1135392, Avg. loss: 0.664949
Total training time: 0.21 seconds.
-- Epoch 97
Norm: 2.58, NNZs: 23, Bias: 1.553331, T: 1147219, Avg. loss: 0.664097
Total training time: 0.21 seconds.
-- Epoch 98
Norm: 2.61, NNZs: 23, Bias: 1.578511, T: 1159046, Avg. loss: 0.664616
Total training time: 0.21 seconds.
-- Epoch 99
Norm: 2.63, NNZs: 23, Bias: 1.502266, T: 1170873, Avg. loss: 0.664936
Total training time: 0.22 seconds.
-- Epoch 100
Norm: 2.61, NNZs: 23, Bias: 1.563425, T: 1182700, Avg. loss: 0.664488
Total training time: 0.22 seconds.
-- Epoch 101
Norm: 2.59, NNZs: 23, Bias: 1.540085, T: 1194527, Avg. loss: 0

Norm: 2.59, NNZs: 23, Bias: 1.494293, T: 2152514, Avg. loss: 0.663156
Total training time: 0.40 seconds.
-- Epoch 183
Norm: 2.58, NNZs: 23, Bias: 1.491220, T: 2164341, Avg. loss: 0.662428
Total training time: 0.40 seconds.
-- Epoch 184
Norm: 2.61, NNZs: 23, Bias: 1.458273, T: 2176168, Avg. loss: 0.663046
Total training time: 0.41 seconds.
-- Epoch 185
Norm: 2.61, NNZs: 23, Bias: 1.453857, T: 2187995, Avg. loss: 0.663110
Total training time: 0.41 seconds.
-- Epoch 186
Norm: 2.58, NNZs: 23, Bias: 1.488091, T: 2199822, Avg. loss: 0.662547
Total training time: 0.41 seconds.
-- Epoch 187
Norm: 2.59, NNZs: 23, Bias: 1.486742, T: 2211649, Avg. loss: 0.662581
Total training time: 0.41 seconds.
-- Epoch 188
Norm: 2.61, NNZs: 23, Bias: 1.439221, T: 2223476, Avg. loss: 0.662838
Total training time: 0.42 seconds.
-- Epoch 189
Norm: 2.57, NNZs: 23, Bias: 1.509507, T: 2235303, Avg. loss: 0.663042
Total training time: 0.42 seconds.
-- Epoch 190
Norm: 2.62, NNZs: 23, Bias: 1.431863, T: 2247130, Avg. l

Norm: 2.58, NNZs: 23, Bias: 1.446616, T: 3193290, Avg. loss: 0.662207
Total training time: 0.60 seconds.
-- Epoch 271
Norm: 2.57, NNZs: 23, Bias: 1.454470, T: 3205117, Avg. loss: 0.662460
Total training time: 0.61 seconds.
-- Epoch 272
Norm: 2.61, NNZs: 23, Bias: 1.397134, T: 3216944, Avg. loss: 0.662255
Total training time: 0.61 seconds.
-- Epoch 273
Norm: 2.59, NNZs: 23, Bias: 1.412698, T: 3228771, Avg. loss: 0.662412
Total training time: 0.61 seconds.
-- Epoch 274
Norm: 2.59, NNZs: 23, Bias: 1.437916, T: 3240598, Avg. loss: 0.662399
Total training time: 0.61 seconds.
-- Epoch 275
Norm: 2.59, NNZs: 23, Bias: 1.445395, T: 3252425, Avg. loss: 0.662500
Total training time: 0.62 seconds.
-- Epoch 276
Norm: 2.62, NNZs: 23, Bias: 1.381618, T: 3264252, Avg. loss: 0.662356
Total training time: 0.62 seconds.
-- Epoch 277
Norm: 2.58, NNZs: 23, Bias: 1.444873, T: 3276079, Avg. loss: 0.662436
Total training time: 0.62 seconds.
-- Epoch 278
Norm: 2.59, NNZs: 23, Bias: 1.410303, T: 3287906, Avg. l

Norm: 2.59, NNZs: 23, Bias: 1.383747, T: 4281374, Avg. loss: 0.662028
Total training time: 0.81 seconds.
-- Epoch 363
Norm: 2.59, NNZs: 23, Bias: 1.395270, T: 4293201, Avg. loss: 0.662113
Total training time: 0.81 seconds.
-- Epoch 364
Norm: 2.61, NNZs: 23, Bias: 1.359195, T: 4305028, Avg. loss: 0.662045
Total training time: 0.81 seconds.
-- Epoch 365
Norm: 2.59, NNZs: 23, Bias: 1.389097, T: 4316855, Avg. loss: 0.662198
Total training time: 0.81 seconds.
-- Epoch 366
Norm: 2.59, NNZs: 23, Bias: 1.398507, T: 4328682, Avg. loss: 0.662162
Total training time: 0.82 seconds.
-- Epoch 367
Norm: 2.58, NNZs: 23, Bias: 1.415518, T: 4340509, Avg. loss: 0.662105
Total training time: 0.82 seconds.
-- Epoch 368
Norm: 2.58, NNZs: 23, Bias: 1.401549, T: 4352336, Avg. loss: 0.661974
Total training time: 0.82 seconds.
-- Epoch 369
Norm: 2.60, NNZs: 23, Bias: 1.372658, T: 4364163, Avg. loss: 0.662356
Total training time: 0.82 seconds.
-- Epoch 370
Norm: 2.59, NNZs: 23, Bias: 1.389836, T: 4375990, Avg. l

...

Norm: 2.58, NNZs: 23, Bias: 1.227662, T: 55705170, Avg. loss: 0.661136
Total training time: 11.08 seconds.
-- Epoch 4711
Norm: 2.58, NNZs: 23, Bias: 1.230118, T: 55716997, Avg. loss: 0.661176
Total training time: 11.08 seconds.
-- Epoch 4712
Norm: 2.58, NNZs: 23, Bias: 1.231423, T: 55728824, Avg. loss: 0.661164
Total training time: 11.09 seconds.
-- Epoch 4713
Norm: 2.58, NNZs: 23, Bias: 1.233147, T: 55740651, Avg. loss: 0.661165
Total training time: 11.09 seconds.
-- Epoch 4714
Norm: 2.58, NNZs: 23, Bias: 1.232680, T: 55752478, Avg. loss: 0.661183
Total training time: 11.09 seconds.
-- Epoch 4715
Norm: 2.58, NNZs: 23, Bias: 1.230768, T: 55764305, Avg. loss: 0.661177
Total training time: 11.10 seconds.
-- Epoch 4716
Norm: 2.58, NNZs: 23, Bias: 1.230772, T: 55776132, Avg. loss: 0.661178
Total training time: 11.10 seconds.
-- Epoch 4717
Norm: 2.58, NNZs: 23, Bias: 1.230349, T: 55787959, Avg. loss: 0.661176
Total training time: 11.10 seconds.
-- Epoch 4718
Norm: 2.58, NNZs: 23, Bias: 1.23

Norm: 2.58, NNZs: 23, Bias: 1.231709, T: 56686811, Avg. loss: 0.661153
Total training time: 11.28 seconds.
-- Epoch 4794
Norm: 2.58, NNZs: 23, Bias: 1.234167, T: 56698638, Avg. loss: 0.661141
Total training time: 11.29 seconds.
-- Epoch 4795
Norm: 2.58, NNZs: 23, Bias: 1.231826, T: 56710465, Avg. loss: 0.661179
Total training time: 11.29 seconds.
-- Epoch 4796
Norm: 2.58, NNZs: 23, Bias: 1.234565, T: 56722292, Avg. loss: 0.661148
Total training time: 11.29 seconds.
-- Epoch 4797
Norm: 2.58, NNZs: 23, Bias: 1.227380, T: 56734119, Avg. loss: 0.661157
Total training time: 11.30 seconds.
-- Epoch 4798
Norm: 2.58, NNZs: 23, Bias: 1.230716, T: 56745946, Avg. loss: 0.661176
Total training time: 11.30 seconds.
-- Epoch 4799
Norm: 2.58, NNZs: 23, Bias: 1.230598, T: 56757773, Avg. loss: 0.661179
Total training time: 11.30 seconds.
-- Epoch 4800
Norm: 2.58, NNZs: 23, Bias: 1.225366, T: 56769600, Avg. loss: 0.661138
Total training time: 11.30 seconds.
-- Epoch 4801
Norm: 2.58, NNZs: 23, Bias: 1.23

Total training time: 11.48 seconds.
-- Epoch 4874
Norm: 2.58, NNZs: 23, Bias: 1.229562, T: 57644798, Avg. loss: 0.661170
Total training time: 11.49 seconds.
-- Epoch 4875
Norm: 2.58, NNZs: 23, Bias: 1.229382, T: 57656625, Avg. loss: 0.661168
Total training time: 11.49 seconds.
-- Epoch 4876
Norm: 2.58, NNZs: 23, Bias: 1.226989, T: 57668452, Avg. loss: 0.661158
Total training time: 11.49 seconds.
-- Epoch 4877
Norm: 2.58, NNZs: 23, Bias: 1.231177, T: 57680279, Avg. loss: 0.661160
Total training time: 11.49 seconds.
-- Epoch 4878
Norm: 2.58, NNZs: 23, Bias: 1.231296, T: 57692106, Avg. loss: 0.661173
Total training time: 11.50 seconds.
-- Epoch 4879
Norm: 2.58, NNZs: 23, Bias: 1.230658, T: 57703933, Avg. loss: 0.661175
Total training time: 11.50 seconds.
-- Epoch 4880
Norm: 2.58, NNZs: 23, Bias: 1.225486, T: 57715760, Avg. loss: 0.661137
Total training time: 11.50 seconds.
-- Epoch 4881
Norm: 2.58, NNZs: 23, Bias: 1.229335, T: 57727587, Avg. loss: 0.661153
Total training time: 11.50 secon

Norm: 2.58, NNZs: 23, Bias: 1.231723, T: 58626439, Avg. loss: 0.661155
Total training time: 11.68 seconds.
-- Epoch 4958
Norm: 2.58, NNZs: 23, Bias: 1.231187, T: 58638266, Avg. loss: 0.661167
Total training time: 11.69 seconds.
-- Epoch 4959
Norm: 2.58, NNZs: 23, Bias: 1.224518, T: 58650093, Avg. loss: 0.661136
Total training time: 11.69 seconds.
-- Epoch 4960
Norm: 2.58, NNZs: 23, Bias: 1.227121, T: 58661920, Avg. loss: 0.661181
Total training time: 11.69 seconds.
-- Epoch 4961
Norm: 2.58, NNZs: 23, Bias: 1.225821, T: 58673747, Avg. loss: 0.661157
Total training time: 11.70 seconds.
-- Epoch 4962
Norm: 2.58, NNZs: 23, Bias: 1.231148, T: 58685574, Avg. loss: 0.661141
Total training time: 11.70 seconds.
-- Epoch 4963
Norm: 2.58, NNZs: 23, Bias: 1.233839, T: 58697401, Avg. loss: 0.661134
Total training time: 11.70 seconds.
-- Epoch 4964
Norm: 2.58, NNZs: 23, Bias: 1.230042, T: 58709228, Avg. loss: 0.661183
Total training time: 11.70 seconds.
-- Epoch 4965
Norm: 2.58, NNZs: 23, Bias: 1.22

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=5000, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=1, warm_start=False)

In [29]:
from sklearn.metrics import accuracy_score

In [30]:
predictions = clf.predict(X_test)

In [31]:
accuracy_score(y_true=y_test, y_pred=predictions)

0.56814338856949609

In [32]:
from sklearn.ensemble import GradientBoostingClassifier

In [33]:
gb_clf = GradientBoostingClassifier(n_estimators=400, verbose=1)

In [34]:
gb_clf.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1           1.3738            2.98s
         2           1.3660            2.97s
         3           1.3594            2.98s
         4           1.3541            2.92s
         5           1.3495            2.88s
         6           1.3457            2.89s
         7           1.3426            2.89s
         8           1.3399            2.84s
         9           1.3377            2.82s
        10           1.3358            2.79s
        20           1.3267            2.63s
        30           1.3228            2.35s
        40           1.3206            2.26s
        50           1.3191            2.17s
        60           1.3179            2.07s
        70           1.3171            2.00s
        80           1.3165            1.94s
        90           1.3159            1.88s
       100           1.3154            1.83s
       200           1.3114            1.18s
       300           1.3091            0.60s
       40

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=400,
              presort='auto', random_state=None, subsample=1.0, verbose=1,
              warm_start=False)

In [35]:
predictions = gb_clf.predict(X_test)
accuracy_score(y_true=y_test, y_pred=predictions)

0.56070341562394321

In [36]:
from sklearn.ensemble import AdaBoostClassifier
ab_clf = AdaBoostClassifier(n_estimators=1000)
ab_clf.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=1000, random_state=None)

In [37]:
predictions = ab_clf.predict(X_test)
accuracy_score(y_true=y_test, y_pred=predictions)

0.57321609739600943

In [38]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1, verbose=1)
rf_clf.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    1.6s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False)

In [39]:
predictions = rf_clf.predict(X_test)
accuracy_score(y_true=y_test, y_pred=predictions)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    0.3s finished


0.55393980385525876

In [40]:
modified_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14784 entries, 0 to 14783
Data columns (total 10 columns):
group_tropes               14784 non-null object
sentence_pattern_parsed    14558 non-null object
genres                     14784 non-null object
contains_characters        14784 non-null object
separate_trope_words       14752 non-null object
trope                      14784 non-null object
sentence_subjectivity      14784 non-null object
sentence_polarity          14784 non-null object
sentence_modality          14784 non-null object
spoiler                    14784 non-null bool
dtypes: bool(1), object(9)
memory usage: 1.0+ MB


In [42]:
feature_cols = [modified_train_data.group_tropes, modified_train_data.sentence_pattern_parsed,
               modified_train_data.genres]
df_cat = pd.DataFrame(reduce(lambda left,right: left.str.cat(right, sep=" "), feature_cols))

In [43]:
df_cat[0:10]

,group_tropes
0,Other see genre_Comedy
1,Other repeat rare NNP show occasional genre_Adventure Sci-Fi
2,Other bad romantic NNP avoid interested work genre_Adventure Sci-Fi
3,Other NNP drive NNP NNP NNP compact remodelled genre_Adventure Sci-Fi
4,narrative_device_trope discover NNP freeze NNP genre_Adventure Sci-Fi
5,Other NNP work download genre_Action Adventure Drama Sci-Fi
6,Other NNP newly-independent NNP NNP hope use resolve agricultural genre_Action Adventure Drama Sci-Fi
7,drama_trope state NNP offer NNP genre_Action Adventure Drama Sci-Fi
8,Other NNP NNP animated perform modernized classic genre_Comedy Drama Family Sci-Fi
9,Other comic tend play give strange technological seem inspire animated original would include set NNP fram...


In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [45]:
df_cat = df_cat.fillna("NA")

In [46]:
X_train, X_test, y_train, y_test = train_test_split(df_cat, labels, test_size=0.2)

In [47]:
X_train.shape

(11827, 1)

In [48]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2))

In [49]:
tf.fit(X_train.group_tropes)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [50]:
X_train_tf = tf.transform(X_train.group_tropes)

In [51]:
X_test_tf = tf.transform(X_test.group_tropes)

In [52]:
rf_clf.fit(X_train_tf, y_train)

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   30.6s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   56.1s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  1.2min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False)

In [53]:
predictions = rf_clf.predict(X_test_tf)
accuracy_score(y_true=y_test, y_pred=predictions)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    0.5s finished


0.70476834629692253

In [205]:
clf.fit(X_train_tf, y_train)

-- Epoch 1
Norm: 37.25, NNZs: 39075, Bias: -0.262131, T: 11827, Avg. loss: 0.616969
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 35.85, NNZs: 39075, Bias: -0.185062, T: 23654, Avg. loss: 0.514030
Total training time: 0.01 seconds.
-- Epoch 3
Norm: 35.30, NNZs: 39075, Bias: -0.241944, T: 35481, Avg. loss: 0.500590
Total training time: 0.01 seconds.
-- Epoch 4
Norm: 35.06, NNZs: 39075, Bias: -0.262926, T: 47308, Avg. loss: 0.495181
Total training time: 0.01 seconds.
-- Epoch 5
Norm: 34.86, NNZs: 39075, Bias: -0.262890, T: 59135, Avg. loss: 0.491997
Total training time: 0.01 seconds.
-- Epoch 6
Norm: 34.74, NNZs: 39075, Bias: -0.258675, T: 70962, Avg. loss: 0.490383
Total training time: 0.02 seconds.
-- Epoch 7
Norm: 34.69, NNZs: 39075, Bias: -0.265056, T: 82789, Avg. loss: 0.488719
Total training time: 0.02 seconds.
-- Epoch 8
Norm: 34.64, NNZs: 39075, Bias: -0.262574, T: 94616, Avg. loss: 0.487784
Total training time: 0.02 seconds.
-- Epoch 9
Norm: 34.52, NNZs: 39075, Bias: -0.27

Norm: 34.09, NNZs: 39075, Bias: -0.266301, T: 839717, Avg. loss: 0.482489
Total training time: 0.20 seconds.
-- Epoch 72
Norm: 34.09, NNZs: 39075, Bias: -0.266561, T: 851544, Avg. loss: 0.482457
Total training time: 0.20 seconds.
-- Epoch 73
Norm: 34.09, NNZs: 39075, Bias: -0.265807, T: 863371, Avg. loss: 0.482447
Total training time: 0.21 seconds.
-- Epoch 74
Norm: 34.09, NNZs: 39075, Bias: -0.264023, T: 875198, Avg. loss: 0.482425
Total training time: 0.21 seconds.
-- Epoch 75
Norm: 34.09, NNZs: 39075, Bias: -0.265375, T: 887025, Avg. loss: 0.482445
Total training time: 0.22 seconds.
-- Epoch 76
Norm: 34.09, NNZs: 39075, Bias: -0.266803, T: 898852, Avg. loss: 0.482420
Total training time: 0.22 seconds.
-- Epoch 77
Norm: 34.09, NNZs: 39075, Bias: -0.266836, T: 910679, Avg. loss: 0.482408
Total training time: 0.22 seconds.
-- Epoch 78
Norm: 34.09, NNZs: 39075, Bias: -0.266537, T: 922506, Avg. loss: 0.482401
Total training time: 0.23 seconds.
-- Epoch 79
Norm: 34.08, NNZs: 39075, Bias: 

-- Epoch 141
Norm: 34.05, NNZs: 39075, Bias: -0.265771, T: 1667607, Avg. loss: 0.482161
Total training time: 0.40 seconds.
-- Epoch 142
Norm: 34.05, NNZs: 39075, Bias: -0.266335, T: 1679434, Avg. loss: 0.482154
Total training time: 0.41 seconds.
-- Epoch 143
Norm: 34.05, NNZs: 39075, Bias: -0.266552, T: 1691261, Avg. loss: 0.482164
Total training time: 0.41 seconds.
-- Epoch 144
Norm: 34.05, NNZs: 39075, Bias: -0.265992, T: 1703088, Avg. loss: 0.482159
Total training time: 0.42 seconds.
-- Epoch 145
Norm: 34.05, NNZs: 39075, Bias: -0.265970, T: 1714915, Avg. loss: 0.482153
Total training time: 0.42 seconds.
-- Epoch 146
Norm: 34.05, NNZs: 39075, Bias: -0.265958, T: 1726742, Avg. loss: 0.482148
Total training time: 0.42 seconds.
-- Epoch 147
Norm: 34.05, NNZs: 39075, Bias: -0.266384, T: 1738569, Avg. loss: 0.482150
Total training time: 0.42 seconds.
-- Epoch 148
Norm: 34.05, NNZs: 39075, Bias: -0.266083, T: 1750396, Avg. loss: 0.482157
Total training time: 0.43 seconds.
-- Epoch 149
Nor

Norm: 34.03, NNZs: 39075, Bias: -0.265972, T: 2507324, Avg. loss: 0.482074
Total training time: 0.60 seconds.
-- Epoch 213
Norm: 34.03, NNZs: 39075, Bias: -0.266062, T: 2519151, Avg. loss: 0.482067
Total training time: 0.61 seconds.
-- Epoch 214
Norm: 34.03, NNZs: 39075, Bias: -0.266057, T: 2530978, Avg. loss: 0.482057
Total training time: 0.61 seconds.
-- Epoch 215
Norm: 34.03, NNZs: 39075, Bias: -0.265926, T: 2542805, Avg. loss: 0.482058
Total training time: 0.61 seconds.
-- Epoch 216
Norm: 34.03, NNZs: 39075, Bias: -0.266136, T: 2554632, Avg. loss: 0.482056
Total training time: 0.62 seconds.
-- Epoch 217
Norm: 34.03, NNZs: 39075, Bias: -0.266255, T: 2566459, Avg. loss: 0.482060
Total training time: 0.62 seconds.
-- Epoch 218
Norm: 34.03, NNZs: 39075, Bias: -0.266294, T: 2578286, Avg. loss: 0.482062
Total training time: 0.62 seconds.
-- Epoch 219
Norm: 34.03, NNZs: 39075, Bias: -0.266134, T: 2590113, Avg. loss: 0.482058
Total training time: 0.63 seconds.
-- Epoch 220
Norm: 34.03, NNZ

Total training time: 0.80 seconds.
-- Epoch 280
Norm: 34.02, NNZs: 39075, Bias: -0.266225, T: 3311560, Avg. loss: 0.482022
Total training time: 0.81 seconds.
-- Epoch 281
Norm: 34.03, NNZs: 39075, Bias: -0.266038, T: 3323387, Avg. loss: 0.482019
Total training time: 0.81 seconds.
-- Epoch 282
Norm: 34.03, NNZs: 39075, Bias: -0.266003, T: 3335214, Avg. loss: 0.482019
Total training time: 0.81 seconds.
-- Epoch 283
Norm: 34.02, NNZs: 39075, Bias: -0.266210, T: 3347041, Avg. loss: 0.482014
Total training time: 0.82 seconds.
-- Epoch 284
Norm: 34.02, NNZs: 39075, Bias: -0.266032, T: 3358868, Avg. loss: 0.482018
Total training time: 0.82 seconds.
-- Epoch 285
Norm: 34.02, NNZs: 39075, Bias: -0.266037, T: 3370695, Avg. loss: 0.482020
Total training time: 0.82 seconds.
-- Epoch 286
Norm: 34.02, NNZs: 39075, Bias: -0.265936, T: 3382522, Avg. loss: 0.482015
Total training time: 0.83 seconds.
-- Epoch 287
Norm: 34.02, NNZs: 39075, Bias: -0.265939, T: 3394349, Avg. loss: 0.482014
Total training t

...

Norm: 34.00, NNZs: 39075, Bias: -0.266205, T: 54073044, Avg. loss: 0.481891
Total training time: 14.34 seconds.
-- Epoch 4573
Norm: 34.00, NNZs: 39075, Bias: -0.266206, T: 54084871, Avg. loss: 0.481891
Total training time: 14.35 seconds.
-- Epoch 4574
Norm: 34.00, NNZs: 39075, Bias: -0.266206, T: 54096698, Avg. loss: 0.481892
Total training time: 14.35 seconds.
-- Epoch 4575
Norm: 34.00, NNZs: 39075, Bias: -0.266206, T: 54108525, Avg. loss: 0.481892
Total training time: 14.35 seconds.
-- Epoch 4576
Norm: 34.00, NNZs: 39075, Bias: -0.266206, T: 54120352, Avg. loss: 0.481891
Total training time: 14.36 seconds.
-- Epoch 4577
Norm: 34.00, NNZs: 39075, Bias: -0.266205, T: 54132179, Avg. loss: 0.481891
Total training time: 14.36 seconds.
-- Epoch 4578
Norm: 34.00, NNZs: 39075, Bias: -0.266205, T: 54144006, Avg. loss: 0.481891
Total training time: 14.36 seconds.
-- Epoch 4579
Norm: 34.00, NNZs: 39075, Bias: -0.266206, T: 54155833, Avg. loss: 0.481891
Total training time: 14.37 seconds.
-- Epo

-- Epoch 4695
Norm: 34.00, NNZs: 39075, Bias: -0.266200, T: 55527765, Avg. loss: 0.481891
Total training time: 14.75 seconds.
-- Epoch 4696
Norm: 34.00, NNZs: 39075, Bias: -0.266199, T: 55539592, Avg. loss: 0.481891
Total training time: 14.75 seconds.
-- Epoch 4697
Norm: 34.00, NNZs: 39075, Bias: -0.266199, T: 55551419, Avg. loss: 0.481891
Total training time: 14.76 seconds.
-- Epoch 4698
Norm: 34.00, NNZs: 39075, Bias: -0.266199, T: 55563246, Avg. loss: 0.481891
Total training time: 14.76 seconds.
-- Epoch 4699
Norm: 34.00, NNZs: 39075, Bias: -0.266199, T: 55575073, Avg. loss: 0.481891
Total training time: 14.76 seconds.
-- Epoch 4700
Norm: 34.00, NNZs: 39075, Bias: -0.266199, T: 55586900, Avg. loss: 0.481891
Total training time: 14.76 seconds.
-- Epoch 4701
Norm: 34.00, NNZs: 39075, Bias: -0.266199, T: 55598727, Avg. loss: 0.481891
Total training time: 14.77 seconds.
-- Epoch 4702
Norm: 34.00, NNZs: 39075, Bias: -0.266200, T: 55610554, Avg. loss: 0.481891
Total training time: 14.77 s

Norm: 34.00, NNZs: 39075, Bias: -0.266207, T: 56947005, Avg. loss: 0.481891
Total training time: 15.15 seconds.
-- Epoch 4816
Norm: 34.00, NNZs: 39075, Bias: -0.266207, T: 56958832, Avg. loss: 0.481891
Total training time: 15.15 seconds.
-- Epoch 4817
Norm: 34.00, NNZs: 39075, Bias: -0.266207, T: 56970659, Avg. loss: 0.481891
Total training time: 15.16 seconds.
-- Epoch 4818
Norm: 34.00, NNZs: 39075, Bias: -0.266206, T: 56982486, Avg. loss: 0.481891
Total training time: 15.16 seconds.
-- Epoch 4819
Norm: 34.00, NNZs: 39075, Bias: -0.266206, T: 56994313, Avg. loss: 0.481891
Total training time: 15.16 seconds.
-- Epoch 4820
Norm: 34.00, NNZs: 39075, Bias: -0.266206, T: 57006140, Avg. loss: 0.481891
Total training time: 15.17 seconds.
-- Epoch 4821
Norm: 34.00, NNZs: 39075, Bias: -0.266206, T: 57017967, Avg. loss: 0.481891
Total training time: 15.17 seconds.
-- Epoch 4822
Norm: 34.00, NNZs: 39075, Bias: -0.266206, T: 57029794, Avg. loss: 0.481891
Total training time: 15.17 seconds.
-- Epo

Norm: 34.00, NNZs: 39075, Bias: -0.266201, T: 58318937, Avg. loss: 0.481891
Total training time: 15.55 seconds.
-- Epoch 4932
Norm: 34.00, NNZs: 39075, Bias: -0.266201, T: 58330764, Avg. loss: 0.481890
Total training time: 15.56 seconds.
-- Epoch 4933
Norm: 34.00, NNZs: 39075, Bias: -0.266201, T: 58342591, Avg. loss: 0.481891
Total training time: 15.56 seconds.
-- Epoch 4934
Norm: 34.00, NNZs: 39075, Bias: -0.266201, T: 58354418, Avg. loss: 0.481891
Total training time: 15.56 seconds.
-- Epoch 4935
Norm: 34.00, NNZs: 39075, Bias: -0.266201, T: 58366245, Avg. loss: 0.481890
Total training time: 15.57 seconds.
-- Epoch 4936
Norm: 34.00, NNZs: 39075, Bias: -0.266202, T: 58378072, Avg. loss: 0.481891
Total training time: 15.57 seconds.
-- Epoch 4937
Norm: 34.00, NNZs: 39075, Bias: -0.266203, T: 58389899, Avg. loss: 0.481891
Total training time: 15.57 seconds.
-- Epoch 4938
Norm: 34.00, NNZs: 39075, Bias: -0.266203, T: 58401726, Avg. loss: 0.481891
Total training time: 15.58 seconds.
-- Epo

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=5000, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=1, warm_start=False)

In [206]:
predictions = clf.predict(X_test_tf)
accuracy_score(y_true=y_test, y_pred=predictions)

0.69834291511667235

In [207]:
gb_clf.fit(X_train_tf, y_train)

      Iter       Train Loss   Remaining Time 
         1           1.3623           30.19s
         2           1.3456           29.46s
         3           1.3315           28.74s
         4           1.3199           28.66s
         5           1.3093           28.72s
         6           1.3005           28.71s
         7           1.2925           28.37s
         8           1.2852           28.10s
         9           1.2791           27.87s
        10           1.2731           27.82s
        20           1.2284           27.15s
        30           1.2013           27.13s
        40           1.1831           26.13s
        50           1.1690           25.16s
        60           1.1571           24.27s
        70           1.1473           23.42s
        80           1.1395           22.69s
        90           1.1319           21.94s
       100           1.1249           21.19s
       200           1.0730           13.88s
       300           1.0377            6.86s
       40

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=400,
              presort='auto', random_state=None, subsample=1.0, verbose=1,
              warm_start=False)

In [208]:
predictions = gb_clf.predict(X_test_tf)
accuracy_score(y_true=y_test, y_pred=predictions)

0.69259384511329047